In [ ]:
import requests
from bs4 import BeautifulSoup # using BeautifulSoup when extracting links from HTML pages
import pandas as pd # using pandas when extracting tables from HTML pages
import time

In [ ]:
years = list(range(2024, 2018, -1)) #(2024, 2018, -1)
years

In [ ]:
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
main_url = "https://fbref.com"

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"{main_url}{l}" for l in links]
    
    previous_season = soup.select('a.prev')[0].get("href")
    standings_url = f"{main_url}{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        
        team_data = requests.get(team_url)
        matches = pd.read_html(team_data.text, match="Scores & Fixtures")[0]
        
        soup = BeautifulSoup(team_data.text)
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        shooting_link = links[0]
        shooting_url = f"{main_url}{shooting_link}"
        shooting_data = requests.get(shooting_url)
        shooting = pd.read_html(shooting_data.text, match = "Shooting")
        
        shooting_for = shooting[0]
        shooting_agst = shooting[1]
        shooting_for.columns = shooting_for.columns.droplevel()
        shooting_agst.columns = shooting_agst.columns.droplevel()
        
        old_columns = shooting_agst.columns.tolist()
        new_columns = {old_col: "Opp" + old_col for old_col in old_columns}
        shooting_agst.rename(columns = new_columns, inplace = True)
        shooting_agst.rename(columns = {'OppDate': 'Date'}, inplace = True)
        
        try:
            team_all_data = matches.merge(shooting_for[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on = "Date")
            team_all_data = team_all_data.merge(shooting_agst[["Date", "OppSh", "OppSoT", "OppDist", "OppFK", "OppPK", "OppPKatt"]], on = "Date")
        except ValueError:
            continue
            
        team_all_data["Season"] = year
        team_all_data["Team"] = team_name
        
        team_pl_data = team_all_data[team_all_data["Comp"] == "Premier League"]
        
        all_matches.append(team_pl_data)
        time.sleep(1)

In [ ]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("PL_Matches.csv")

In [ ]:
match_df

In [ ]:
#end here

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
data = requests.get(standings_url)

In [ ]:
data.text

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
standings_table = soup.select('table.stats_table')[0]

In [ ]:
links = standings_table.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if '/squads/' in l]
links

In [ ]:
main_url = "https://fbref.com"
team_urls = [f"{main_url}{l}" for l in links]
team_urls

In [ ]:
team_url = team_urls[1] #Man City is currently 2nd

In [ ]:
team_data = requests.get(team_url)

In [ ]:
team_data.text

In [ ]:
matches = pd.read_html(team_data.text, match="Scores & Fixtures")

In [ ]:
matches = matches[0]
matches

In [ ]:
soup = BeautifulSoup(team_data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l]
links

In [ ]:
shooting_link = links[0]
shooting_data = requests.get(f"{main_url}{shooting_link}")

In [ ]:
shooting_data.text

In [ ]:
shooting = pd.read_html(shooting_data.text, match = "Shooting")

In [ ]:
shooting_for = shooting[0]
shooting_agst = shooting[1]

In [ ]:
shooting_for.columns = shooting_for.columns.droplevel()
shooting_agst.columns = shooting_agst.columns.droplevel()

In [ ]:
shooting_for

In [ ]:
shooting_agst

In [ ]:
old_columns = shooting_agst.columns.tolist()
new_columns = {old_col: "Opp" + old_col for old_col in old_columns}
shooting_agst.rename(columns = new_columns, inplace = True)
shooting_agst.rename(columns = {'OppDate': 'Date'}, inplace = True)

In [ ]:
team_all_data = matches.merge(shooting_for[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt", "xG", "npxG"]], on = "Date")
team_all_data = team_all_data.merge(shooting_agst[["Date", "OppSh", "OppSoT", "OppDist", "OppFK", "OppPK", "OppPKatt", "OppxG", "OppnpxG"]], on = "Date")

In [ ]:
team_all_data